# MATSim Read-Write

This notebook is an introduction to the PAM MATSim read-write methods. It has four parts:

1. [Read](#read-matsim-format)
2. [Write](#write-matsim-format)
3. [Experienced Plans](#experienced-plans)
4. [Streaming](#streaming)

In [1]:
import os

from pam import read

## Read MATSim Format

PAM can read from MATSim formatted xml using `pam.read.read_matsim`. `read_matsim` will except xml or gzipped xml (`.xml.gz`). It supports multiple additional options for your use-case (read the docs).

In [2]:
population = read.read_matsim(plans_path=os.path.join("data", "example_data", "example_plans.xml"))
population.stats

{'num_households': 51,
 'num_people': 51,
 'num_activities': 153,
 'num_legs': 102}

## Write MATSim Format

PAM can write to MATSim formatted xml using `pam.write.write_matsim`.

In [3]:
from pam import write

write.write_matsim(population, plans_path=os.path.join("tmp", "plans.xml"))

## Experienced Plans

PAM supports MATSim experienced plans:

- unselected plans
- missing person attributes (attributes can be retrieved from regular MATSim plans)
- leg attributes
- leg routes

The `read_matsim` and `write_matsim` functions support experienced plans. Reading, modifying and writing experienced plans is a useful way of building MATSim scenarios without loosing too much information from a previous scenario.


## Streaming

Low memory applications can be created by streaming MATSim plans from `read.stream_matsim_persons` into your application logic and then back to disk using `write.Writer`. For example:

In [4]:
from pam.read import stream_matsim_persons
from pam.write import Writer

read_path = os.path.join("data", "example_data", "example_plans.xml")
write_path = os.path.join("tmp", "plans.xml")

with Writer(
    path=write_path, keep_non_selected=True, coordinate_reference_system="EPSG:27700"
) as writer:
    for person in stream_matsim_persons(
        plans_path=read_path,
        weight=10,
        simplify_pt_trips=False,
        leg_route=False,
        keep_non_selected=True,
    ):
        # add age attribute
        person.attributes["age"] = "unknown"
        writer.add_person(person)